In [1]:
!pip install transformers
!pip install datasets
!pip install optuna

     |████████████████████████████████| 2.1MB 13.6MB/s 
     |████████████████████████████████| 3.3MB 39.0MB/s 
     |████████████████████████████████| 901kB 55.7MB/s 
     |████████████████████████████████| 225kB 12.1MB/s 
     |████████████████████████████████| 245kB 26.5MB/s 
     |████████████████████████████████| 112kB 23.6MB/s 
     |████████████████████████████████| 296kB 16.9MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 163kB 30.3MB/s 
     |████████████████████████████████| 143kB 35.1MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 112kB 33.7MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=c5ca754ed4ed045368cedf32bbf5ee39eae2761b2faaed2a0a46ea53c38abbe9
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully 

In [2]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-3128fecc-2d8c-c38d-3b4a-d8944d10bb05)


## Fine-tuning a model on a text classification task
This notebook will show to fine-tune one of the 🤗 Transformers model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/).
<br><br>
The **GLUE Benchmark** is a group of nine classification tasks on sentences or pairs of sentences which are:

- **CoLA** (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not. It is a dataset containing sentences labeled grammatically correct or not.
- **MNLI** (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- **MRPC** (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- **QNLI** (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- **QQP** (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- **RTE** (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- **SST-2** (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- **STS-B** (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- **WNLI** (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)
<br><br>

Among them, this notebook will use The **STS-B** Dataset.

In [3]:
import datasets

import random
import pandas as pd
import numpy as np

from IPython.display import display, HTML

import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# datasets : 1.5.0  |  pd : 1.1.5  |  np : 1.19.5  |  transformers : 4.5.1
print(f'datasets : {datasets.__version__}  |  pd : {pd.__version__}  |  np : {np.__version__}  |  transformers : {transformers.__version__}  |  torch : {torch.__version__}')
print('device :', device)

datasets : 1.6.1  |  pd : 1.1.5  |  np : 1.19.5  |  transformers : 4.5.1  |  torch : 1.8.1+cu101
device : cuda


In [4]:
# check execution time for whole code
import time
s_time = time.time()

We will see how to easily load the dataset for each one of those tasks and use the `Trainer` API to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [5]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
task = "stsb"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## 1. Loading the dataset & metric
We will use the 🤗 Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.

In [6]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = datasets.load_dataset("glue", actual_task)
metric = datasets.load_metric('glue', actual_task)

print('\n>>> actual_task :', actual_task)
print('\n>>> type of metric :', type(metric))
print('\n>>> dataset object :', dataset)
print('\n>>> sample data :', dataset['train'][0])

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.




>>> actual_task : stsb

>>> type of metric : <class 'datasets_modules.metrics.glue.e4606ab9804a36bcd5a9cebb2cb65bb14b6ac78ee9e6d5981fa679a495dd55de.glue.Glue'>

>>> dataset object : DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 5749
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1379
    })
})

>>> sample data : {'idx': 0, 'label': 5.0, 'sentence1': 'A plane is taking off.', 'sentence2': 'An air plane is taking off.'}


In [7]:
# show random sample of a dataset
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = random.sample(range(len(dataset)), k=num_examples)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"])

,idx,label,sentence1,sentence2
0,4796,1.0,No deal on fiscal cliff as Obama goes on holiday,"John Boehner: from humble origin, fiscal cliff may be his undoing"
1,2067,3.2,Can he really make mone from google ad sense.,Who makes money off google ad sense?
2,4267,1.0,New Syria opposition chief wants no-strings aid,Syria opposition unites as Israel fires warning shots
3,3574,2.4,last week russia announced that it will suspend participation in the cfe.,nato regrets russia's decision to suspend participation in the cfe.
4,3485,4.6,anatoly sokolov announced-- soviet-built a-135 missile defense system around moscow is obsolete and inefficient.,soviet-built a-135 missile defense system around moscow is obsolete and inefficient.


In [8]:
# compute score with metric
fake_preds = np.random.randint(0, 2, size=(16,))
fake_labels = np.random.randint(0, 2, size=(16,))

fake_preds, fake_labels, metric.compute(predictions=fake_preds, references=fake_labels)

(array([1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1]),
 array([0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1]),
 {'pearson': 0.12598815766974242, 'spearmanr': 0.12598815766974242})

The metric object only computes the proper metrics associated to the task, which are:

- for **CoLA** : Matthews Correlation Coefficient
- for **MNLI**(matched or mismatched) : Accuracy
- for **MRPC** : Accuracy and F1 score
- for **QNLI** : Accuracy
- for **QQP** : Accuracy and F1 score
- for **RTE** : Accuracy
- for **SST-2** : Accuracy
- for **STS-B** : Pearson Correlation Coefficient and Spearman's_Rank_Correlation_Coefficient
- for **WNLI** : Accuracy

## 2. Preprocessing the data


In [9]:
%%time
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)


CPU times: user 302 ms, sys: 22.8 ms, total: 325 ms
Wall time: 3.57 s


In [10]:
# tokenize sample sentences
batch_encoded = tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

print(type(batch_encoded))
for k, v in batch_encoded.items():
  print(k, '-', v)

<class 'transformers.tokenization_utils_base.BatchEncoding'>
input_ids - [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102]
attention_mask - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [11]:
# generate dict for saving names of columns containing the sentences
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

# check if the dict works correctly
sent1_key, sent2_key = task_to_keys[task]
if sent2_key is None:
  print(f'Sentence :', dataset['train'][0][sent1_key])
else:
  print(f'Sentence 1 :', dataset['train'][0][sent1_key])
  print(f'Sentence 2 :', dataset['train'][0][sent2_key])

Sentence 1 : A plane is taking off.
Sentence 2 : An air plane is taking off.


In [12]:
# function to preprocess text - tokenize & truncate
def preprocess(examples):
  if sent2_key is None:
    return tokenizer(examples[sent1_key], truncation=True)
  else:
    return tokenizer(examples[sent1_key], examples[sent2_key], truncation=True)


# check if the preprocessor works properly
for k, v in preprocess(dataset['train'][:5]).items():
  print(f"'{k}' :")
  for lst in v:
    print('\t', lst)

'input_ids' :
	 [101, 1037, 4946, 2003, 2635, 2125, 1012, 102, 2019, 2250, 4946, 2003, 2635, 2125, 1012, 102]
	 [101, 1037, 2158, 2003, 2652, 1037, 2312, 8928, 1012, 102, 1037, 2158, 2003, 2652, 1037, 8928, 1012, 102]
	 [101, 1037, 2158, 2003, 9359, 14021, 5596, 2098, 8808, 2006, 1037, 10733, 1012, 102, 1037, 2158, 2003, 9359, 29022, 8808, 2006, 2019, 4895, 3597, 23461, 10733, 1012, 102]
	 [101, 2093, 2273, 2024, 2652, 7433, 1012, 102, 2048, 2273, 2024, 2652, 7433, 1012, 102]
	 [101, 1037, 2158, 2003, 2652, 1996, 10145, 1012, 102, 1037, 2158, 8901, 2003, 2652, 1996, 10145, 1012, 102]
'attention_mask' :
	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [13]:
%%time
# apply preprocess function to all sentences in dataset
# set batched as True to encode the texts by batches together (about 5x faster)
dataset_encoded = dataset.map(preprocess, batched=True)

for k, v in dataset_encoded['train'][0].items():
  print(f'>>> {k} - {v}')


>>> attention_mask - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
>>> idx - 0
>>> input_ids - [101, 1037, 4946, 2003, 2635, 2125, 1012, 102, 2019, 2250, 4946, 2003, 2635, 2125, 1012, 102]
>>> label - 5.0
>>> sentence1 - A plane is taking off.
>>> sentence2 - An air plane is taking off.
CPU times: user 1.29 s, sys: 24.7 ms, total: 1.31 s
Wall time: 543 ms


## 3. Fine-tuning the model

In [14]:
num_labels = 3 if task.startswith('mnli') else 1 if task=='stsb' else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels).to(device)

print()
model

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [15]:
%%time
# define attributes to customize the training
metric_name = 'pearson' if task=='stsb' else 'matthews_correlation' if task=='cola' else 'accuracy'
args = TrainingArguments(
    output_dir='test-glue', evaluation_strategy='epoch', learning_rate=2e-5, 
    per_device_train_batch_size=batch_size, per_device_eval_batch_size=batch_size,
    num_train_epochs=5, weight_decay=0.01, load_best_model_at_end=True, 
    metric_for_best_model=metric_name,
)

# define function to compute the metrics from the predictions
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  if task=='stsb':
    predictions = predictions[:, 0]
  else:
    predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)


# generate trainer & train model
validation_key = 'validation_mismatched' if task=='mmli-mm' else 'validation_matched' if task=='mmli' else 'validation'
trainer = Trainer(model=model, args=args, 
                  train_dataset=dataset_encoded['train'], eval_dataset=dataset_encoded[validation_key],
                  tokenizer=tokenizer, compute_metrics=compute_metrics)
trainer.train()

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr,Runtime,Samples Per Second
1,No log,0.667706,0.861625,0.865182,1.395500,1074.864000
2,1.002400,0.554968,0.869365,0.867225,1.309700,1145.336000
3,0.412800,0.540362,0.872246,0.869421,1.418800,1057.221000
4,0.412800,0.553549,0.870751,0.868630,1.286900,1165.632000
5,0.257000,0.550112,0.870211,0.867407,1.328900,1128.792000


CPU times: user 3min 12s, sys: 1min 35s, total: 4min 48s
Wall time: 2min 48s


In [16]:
%%time
# evaluate model
trainer.evaluate()

CPU times: user 1.76 s, sys: 535 ms, total: 2.3 s
Wall time: 1.55 s


{'epoch': 5.0,
 'eval_loss': 0.5403618812561035,
 'eval_mem_cpu_alloc_delta': 4096,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 47962112,
 'eval_pearson': 0.8722463755743014,
 'eval_runtime': 1.3186,
 'eval_samples_per_second': 1137.598,
 'eval_spearmanr': 0.8694214632200764}

## 4. Hyperparameter search

In [17]:
%%time
trainer = Trainer(
    model_init=lambda : AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels),
    args=args,
    train_dataset=dataset_encoded['train'].shard(num_shards=10, index=1),  # train with sample train dataset
    eval_dataset=dataset_encoded[validation_key],
    tokenizer=tokenizer, compute_metrics=compute_metrics
)

print('\n\n')
best_run = trainer.hyperparameter_search(n_trials=10, direction='maximize')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

[I 2021-04-28 14:15:25,007] A new study created in memory with name: no-name-e8034de8-6652-42c3-815d-b147db4d5094
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpo

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr,Runtime,Samples Per Second
1,No log,2.435753,0.285022,0.281209,1.354600,1107.331000
2,No log,1.450421,0.629707,0.637030,1.330900,1127.043000
3,No log,0.935764,0.771966,0.771852,1.348400,1112.451000


[I 2021-04-28 14:15:46,328] Trial 0 finished with value: 1.5438177328196434 and parameters: {'learning_rate': 6.993956035863168e-05, 'num_train_epochs': 3, 'seed': 29, 'per_device_train_batch_size': 32}. Best is trial 0 with value: 1.5438177328196434.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model fr

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr,Runtime,Samples Per Second
1,No log,2.298784,0.308847,0.318273,1.233000,1216.517000


...

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr,Runtime,Samples Per Second
1,No log,2.797948,0.131682,0.141449,2.831800,529.693000


[I 2021-04-28 14:20:16,550] Trial 8 pruned. 
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_class

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr,Runtime,Samples Per Second
1,No log,7.407786,0.095884,0.097627,4.107400,365.190000


[I 2021-04-28 14:20:28,488] Trial 9 pruned. 


CPU times: user 5min 34s, sys: 3min 46s, total: 9min 21s
Wall time: 5min 7s


In [18]:
%%time
# generate final trainer
trainer_final = Trainer(
    model_init=lambda : AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels),
    args=args,
    train_dataset=dataset_encoded['train'],  # train with full train dataset
    eval_dataset=dataset_encoded[validation_key],
    tokenizer=tokenizer, compute_metrics=compute_metrics
)

# trian model with best hyper-parameters
print('< best_run >')
for n, v in best_run.hyperparameters.items():
  print(f'>>> {n} : {v}')
  setattr(trainer_final.args, n, v)

print('\n')
trainer_final.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

< best_run >
>>> learning_rate : 2.4337400979185552e-05
>>> num_train_epochs : 5
>>> seed : 10
>>> per_device_train_batch_size : 4




Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Pearson,Spearmanr,Runtime,Samples Per Second
1,0.797700,0.807299,0.854271,0.851399,4.307700,348.211000
2,0.439900,0.629732,0.868090,0.866361,4.207300,356.527000
3,0.248400,0.552358,0.873863,0.869527,4.190700,357.935000
4,0.165100,0.529457,0.874883,0.871436,4.183500,358.549000
5,0.111200,0.533951,0.875012,0.870935,4.237700,353.966000


CPU times: user 23min 55s, sys: 26min 26s, total: 50min 21s
Wall time: 25min 35s


In [19]:
%%time
# evaluate model
trainer_final.evaluate()

CPU times: user 4.23 s, sys: 4 s, total: 8.23 s
Wall time: 4.52 s


{'epoch': 5.0,
 'eval_loss': 0.533950686454773,
 'eval_mem_cpu_alloc_delta': 106496,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 47077376,
 'eval_pearson': 0.8750123576206834,
 'eval_runtime': 4.2202,
 'eval_samples_per_second': 355.436,
 'eval_spearmanr': 0.870934855720869}

In [20]:
# check execution time for whole code
e_time = time.time()
time_elapsed = e_time - s_time
print(f'Total time elapsed : {int(time_elapsed//60)} min {int(time_elapsed%60)} sec')

Total time elapsed : 33 min 58 sec
